In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata

In [2]:
df = pd.read_csv('stock_tweets.csv')
df.head()

,Date,Tweet,Stock Name,Company Name
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc."
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc."
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc."
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc."
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc."


In [3]:
print(df.shape)
print(df.dtypes)

(80793, 4)
Date            object
Tweet           object
Stock Name      object
Company Name    object
dtype: object


In [4]:
# Convert date column to datetime object
df['Date'] = pd.to_datetime(df['Date'])

# Extract date and time components into separate columns
df['time'] = df['Date'].dt.time
df['Date'] = df['Date'].dt.date

df

,Date,Tweet,Stock Name,Company Name,time
0,2022-09-29,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc.",23:41:16
1,2022-09-29,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc.",23:24:43
2,2022-09-29,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc.",23:18:08
3,2022-09-29,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc.",22:40:07
4,2022-09-29,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc.",22:27:05
...,...,...,...,...,...
80788,2021-10-07,Some of the fastest growing tech stocks on the...,XPEV,XPeng Inc.,17:11:57
80789,2021-10-04,"With earnings on the horizon, here is a quick ...",XPEV,XPeng Inc.,17:05:59
80790,2021-10-01,Our record delivery results are a testimony of...,XPEV,XPeng Inc.,04:43:41
80791,2021-10-01,"We delivered 10,412 Smart EVs in Sep 2021, rea...",XPEV,XPeng Inc.,00:03:32


In [5]:
df["sentiment_score"] = ''
df["Negative"] = ''
df["Neutral"] = ''
df["Positive"] = ''
df.head()

,Date,Tweet,Stock Name,Company Name,time,sentiment_score,Negative,Neutral,Positive
0,2022-09-29,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc.",23:41:16,,,,
1,2022-09-29,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc.",23:24:43,,,,
2,2022-09-29,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc.",23:18:08,,,,
3,2022-09-29,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc.",22:40:07,,,,
4,2022-09-29,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc.",22:27:05,,,,


In [6]:
%%time
sentiment_analyzer = SentimentIntensityAnalyzer()
for indx, row in df.T.iteritems():
    try:
        sentence_i = unicodedata.normalize('NFKD', df.loc[indx, 'Tweet'])
        sentence_sentiment = sentiment_analyzer.polarity_scores(sentence_i)
        df.at[indx, 'sentiment_score'] = sentence_sentiment['compound']
        df.at[indx, 'Negative'] = sentence_sentiment['neg']
        df.at[indx, 'Neutral'] = sentence_sentiment['neu']
        df.at[indx, 'Positive'] = sentence_sentiment['pos']
    except TypeError:
        print (df.loc[indexx, 'Tweet'])
        print (indx)
        break

CPU times: user 34 s, sys: 320 ms, total: 34.4 s
Wall time: 35.4 s


In [7]:
df.head()

,Date,Tweet,Stock Name,Company Name,time,sentiment_score,Negative,Neutral,Positive
0,2022-09-29,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc.",23:41:16,0.0772,0.127,0.758,0.115
1,2022-09-29,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc.",23:24:43,0.0,0.0,1.0,0.0
2,2022-09-29,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc.",23:18:08,0.296,0.0,0.951,0.049
3,2022-09-29,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc.",22:40:07,-0.7568,0.273,0.59,0.137
4,2022-09-29,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc.",22:27:05,-0.875,0.526,0.474,0.0


In [8]:
df.to_csv("sentiment-sentiment analyzer.csv")